# MACD-ATR with triangle arbitrage
Documentation and Detailed Explanation: [Link](https://docs.google.com/document/d/1y1DfoK3PE0N_N5b-fMqZMfKD7LEhgRGgBbG8XZx3dC0/edit?usp=sharing)


### Data import

Chosen cryptos:
* BNB/USD: Data includes price, market cap, and volume.
* BTC/USD: Data includes price, market cap, and volume.
* ETH/USD: Data includes price, market cap, and volume.
* SOL/USD: Data includes price, market cap, and volume.
* TRX/USD: Data includes price, market cap, and volume.
* USDC/USD: Data includes price, market cap, and volume.
* USDT/USD: Data includes price, market cap, and volume.
* XRP/USD: Data includes price, market cap, and volume.

In [ ]:
# Load the historical data from the uploaded files
files = {
    'bnb': '/mnt/data/bnb-usd-max.csv',
    'btc': '/mnt/data/btc-usd-max.csv',
    'eth': '/mnt/data/eth-usd-max.csv',
    'sol': '/mnt/data/sol-usd-max.csv',
    'trx': '/mnt/data/trx-usd-max.csv',
    'usdc': '/mnt/data/usdc-usd-max.csv',
    'usdt': '/mnt/data/usdt-usd-max.csv',
    'xrp': '/mnt/data/xrp-usd-max.csv'
}

# Load each file into a dataframe and store in a dictionary
dataframes = {key: pd.read_csv(file_path) for key, file_path in files.items()}

# Check the first few rows of each dataframe to ensure they are loaded correctly
data_samples = {key: df.head() for key, df in dataframes.items()}
data_samples

## Market Insights and Visualization
### Group currencies into high-range and low-range

Cryptocurrencies should be properly categorized into high-range and low-range to ensure readable visualization.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the theme for visualizations
sns.set_theme(style="darkgrid", palette="muted", rc={"axes.facecolor": "#EAF6F5"})

# Define a function to plot historical prices of all cryptocurrencies
def plot_historical_prices(dataframes):
    plt.figure(figsize=(14, 8))
    
    for key, df in dataframes.items():
        plt.plot(df.iloc[:, 0], df.iloc[:, 1], label=key.upper())
    
    plt.title("Historical Prices of Cryptocurrencies")
    plt.xlabel("Date")
    plt.ylabel("Price (USD)")
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Plot the historical prices for all cryptocurrencies
plot_historical_prices(dataframes)

### Plotting price trends for different crypto currencies

In [ ]:
# Separate data into two groups for better visualization

# Group 1: BTC (Higher price range)
high_price_cryptos = ['btc']

# Group 2: Other Cryptos (Lower price range)
low_price_cryptos = ['bnb', 'eth', 'sol', 'trx', 'usdc', 'usdt', 'xrp']

def plot_grouped_historical_prices(dataframes, group, title):
    plt.figure(figsize=(14, 8))
    
    for key in group:
        df = dataframes[key]
        plt.plot(df.iloc[:, 0], df.iloc[:, 1], label=key.upper())
    
    plt.title(title)
    plt.xlabel("Date")
    plt.ylabel("Price (USD)")
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Plot Group 1
plot_grouped_historical_prices(dataframes, high_price_cryptos, "Historical Prices of High-Range Cryptocurrencies")

# Plot Group 2
plot_grouped_historical_prices(dataframes, low_price_cryptos, "Historical Prices of Low-Range Cryptocurrencies")

## LSTM Forecast 
Use LSTM to project future data for strategy testing:

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt

# Prepare data for LSTM
def prepare_lstm_data(df, feature_col, time_steps):
    data = df[feature_col].values.reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)
    
    x, y = [], []
    for i in range(time_steps, len(scaled_data)):
        x.append(scaled_data[i-time_steps:i, 0])
        y.append(scaled_data[i, 0])
        
    x, y = np.array(x), np.array(y)
    x = np.reshape(x, (x.shape[0], x.shape[1], 1))
    
    return x, y, scaler

# Define and train LSTM model
def train_lstm(x_train, y_train, epochs=20, batch_size=32):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)
    
    return model

# Load your data (example: BTC data)
btc_data = pd.read_csv('btc-usd-max.csv')
btc_data['Date'] = pd.to_datetime(btc_data['Date'])
btc_data.set_index('Date', inplace=True)

# Prepare data
time_steps = 60
x, y, scaler = prepare_lstm_data(btc_data, 'Price', time_steps)

# Split data into training and test sets
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Train LSTM model
lstm_model = train_lstm(x_train, y_train, epochs=10)

# Make predictions
predicted_prices = lstm_model.predict(x_test)
predicted_prices = scaler.inverse_transform(predicted_prices)

# Compare predictions with actual prices
actual_prices = btc_data['Price'].values[train_size + time_steps:]

# Plot the actual vs predicted prices
plt.figure(figsize=(14, 6))
plt.plot(actual_prices, color="blue", label="Actual BTC Price")
plt.plot(predicted_prices, color="red", label="Predicted BTC Price")
plt.title("BTC Price Prediction using LSTM")
plt.xlabel("Time")
plt.ylabel("Price")
plt.legend()
plt.tight_layout()
plt.show()


## Strategu Formulation

### ATR Value for BTC

In [ ]:
# Calculate ATR (Average True Range) for BTC
# For simplicity, we use price data as a proxy for High, Low, and Close series in this demo
btc_data['High'] = btc_data['Price']
btc_data['Low'] = btc_data['Price']
btc_data['Close'] = btc_data['Price']

# Calculate ATR
btc_data['ATR'] = calculate_atr(btc_data['High'], btc_data['Low'], btc_data['Close'])

# Plotting ATR for visualization
plt.figure(figsize=(14, 6))
plt.plot(btc_data.index, btc_data['ATR'], label="ATR (Average True Range)", color="blue")
plt.title("ATR for BTC")
plt.xlabel("Date")
plt.ylabel("ATR Value")
plt.legend()
plt.tight_layout()
plt.show()

### Base Strategy: MACD-ATR

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Define the EMA calculation function
def calculate_ema(data, span):
    return data.ewm(span=span, adjust=False).mean()

# Define the ATR calculation function
def calculate_atr(high, low, close, length=13):
    high_low = high - low
    high_close = np.abs(high - close.shift())
    low_close = np.abs(low - close.shift())
    tr = np.maximum(high_low, high_close, low_close)
    atr = tr.rolling(window=length).mean()
    return atr

# Example data loading (replace with your data)
data = pd.read_csv('your_data.csv')  # Replace with your actual data file

# Calculate EMAs
data['EMA13'] = calculate_ema(data['Close'], 13)
data['EMA34'] = calculate_ema(data['Close'], 34)

# Calculate ATR
data['ATR'] = calculate_atr(data['High'], data['Low'], data['Close'], length=13)

# Generate trading signals based on the strategy
data['Signal'] = 0
data['Signal'][data['EMA13'] > data['EMA34']] = 1  # Buy signal when EMA13 > EMA34
data['Signal'][data['EMA13'] < data['EMA34']] = -1  # Sell signal when EMA13 < EMA34

# Calculate stop-loss positions
data['Stop_Loss'] = data['Close'] - data['ATR']

# Set profit target based on a 1:2 return rate
data['Profit_Target'] = data['Close'] + 2 * (data['Close'] - data['Stop_Loss'])

# Backtest the strategy
data['Position'] = data['Signal'].shift()  # Lag the signal to the next day
data['Daily_Return'] = data['Close'].pct_change() * data['Position']

# Calculate cumulative returns
data['Cumulative_Return'] = (1 + data['Daily_Return']).cumprod()

# Plot the results
plt.figure(figsize=(14, 8))
plt.plot(data['Cumulative_Return'], label='Cumulative Return')
plt.title('Trading Strategy Cumulative Return')
plt.legend()
plt.show()

### Triangle Arbitrage Strategy

In [ ]:
def check_arbitrage_opportunity(btc_eth, eth_usdt, btc_usdt):
    implied_btc_usdt = btc_eth * eth_usdt
    arbitrage_opportunity = btc_usdt - implied_btc_usdt
    return arbitrage_opportunity

def execute_arbitrage(btc_amount, btc_eth_rate, eth_usdt_rate, btc_usdt_rate):
    eth_amount = btc_amount / btc_eth_rate
    usdt_amount = eth_amount * eth_usdt_rate
    final_btc_amount = usdt_amount / btc_usdt_rate
    return final_btc_amount

# Assume these are current exchange rates
btc_eth_rate = 0.05
eth_usdt_rate = 2000
btc_usdt_rate = 100000

btc_amount = 1  # Start with 1 BTC

# Check for arbitrage opportunity
arbitrage_profit = check_arbitrage_opportunity(btc_eth_rate, eth_usdt_rate, btc_usdt_rate)

if arbitrage_profit > 0:
    final_btc_amount = execute_arbitrage(btc_amount, btc_eth_rate, eth_usdt_rate, btc_usdt_rate)
    print(f"Arbitrage executed! Initial BTC: {btc_amount}, Final BTC: {final_btc_amount}, Profit: {final_btc_amount - btc_amount} BTC")
else:
    print("No arbitrage opportunity detected.")


## Backtesting strategy

In [ ]:
# Define a function to apply the strategy and backtest it across different cryptocurrencies
def backtest_strategy(data, name):
    data['Position'] = 0  # 1 for buy, -1 for sell
    data['Strategy_Return'] = 0
    data['Cumulative_Return'] = 1  # Start with an initial investment of 1 unit
    
    # Calculate MACD
    macd_line, signal_line, macd_histogram = calculate_macd(data['Price'])
    
    # Calculate ATR
    data['High'] = data['Price']
    data['Low'] = data['Price']
    data['Close'] = data['Price']
    data['ATR'] = calculate_atr(data['High'], data['Low'], data['Close'])
    
    stop_loss_threshold = 0.02  # 2% of ATR
    
    # Backtesting loop
    for i in range(1, len(data)):
        # Buy signal
        if macd_line[i] > signal_line[i] and macd_histogram[i] > 0:
            data['Position'].iloc[i] = 1  # Enter long position
        # Sell signal
        elif macd_line[i] < signal_line[i] and macd_histogram[i] < 0:
            data['Position'].iloc[i] = -1  # Exit position

        # Apply stop-loss based on ATR
        if data['Position'].iloc[i] == 1 and data['Price'].iloc[i] < data['Price'].iloc[i-1] - (stop_loss_threshold * data['ATR'].iloc[i-1]):
            data['Position'].iloc[i] = -1  # Stop loss

        # Calculate Strategy Return
        if data['Position'].iloc[i] == 1:
            data['Strategy_Return'].iloc[i] = (data['Price'].iloc[i] - data['Price'].iloc[i-1]) / data['Price'].iloc[i-1]
        elif data['Position'].iloc[i] == -1:
            data['Strategy_Return'].iloc[i] = 0

        # Calculate Cumulative Return
        data['Cumulative_Return'].iloc[i] = data['Cumulative_Return'].iloc[i-1] * (1 + data['Strategy_Return'].iloc[i])

    return data

# Backtest the strategy on other cryptocurrencies
backtest_results = {}
for crypto in low_price_cryptos:
    crypto_data = dataframes[crypto]
    crypto_data.columns = ['Date', 'Price', 'Market_Cap', 'Volume']
    crypto_data['Date'] = pd.to_datetime(crypto_data['Date'])
    crypto_data.set_index('Date', inplace=True)
    
    backtest_results[crypto] = backtest_strategy(crypto_data, crypto)

# Plot cumulative returns for all backtested cryptocurrencies
plt.figure(figsize=(14, 8))
for crypto, result in backtest_results.items():
    plt.plot(result.index, result['Cumulative_Return'], label=crypto.upper())

plt.title("Backtest: Cumulative Returns of the Strategy on Multiple Cryptocurrencies")
plt.xlabel("Date")
plt.ylabel("Cumulative Return")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Implementing a simple backtesting logic for the strategy on BTC

# Initialize the strategy performance tracking
btc_data['Position'] = 0  # 1 for buy, -1 for sell
btc_data['Strategy_Return'] = 0
btc_data['Cumulative_Return'] = 1  # Start with an initial investment of 1 unit

# Define ATR-based stop loss threshold (e.g., 2% of ATR)
stop_loss_threshold = 0.02

# Backtesting loop
for i in range(1, len(btc_data)):
    # Buy signal
    if btc_macd_line[i] > btc_signal_line[i] and btc_macd_histogram[i] > 0:
        btc_data['Position'].iloc[i] = 1  # Enter long position
    # Sell signal
    elif btc_macd_line[i] < btc_signal_line[i] and btc_macd_histogram[i] < 0:
        btc_data['Position'].iloc[i] = -1  # Exit position
    
    # Apply stop-loss based on ATR
    if btc_data['Position'].iloc[i] == 1 and btc_data['Price'].iloc[i] < btc_data['Price'].iloc[i-1] - (stop_loss_threshold * btc_data['ATR'].iloc[i-1]):
        btc_data['Position'].iloc[i] = -1  # Stop loss
    
    # Calculate Strategy Return
    if btc_data['Position'].iloc[i] == 1:
        btc_data['Strategy_Return'].iloc[i] = (btc_data['Price'].iloc[i] - btc_data['Price'].iloc[i-1]) / btc_data['Price'].iloc[i-1]
    elif btc_data['Position'].iloc[i] == -1:
        btc_data['Strategy_Return'].iloc[i] = 0
    
    # Calculate Cumulative Return
    btc_data['Cumulative_Return'].iloc[i] = btc_data['Cumulative_Return'].iloc[i-1] * (1 + btc_data['Strategy_Return'].iloc[i])

# Plot the cumulative returns for visualization
plt.figure(figsize=(14, 6))
plt.plot(btc_data.index, btc_data['Cumulative_Return'], label="Cumulative Return", color="green")
plt.title("Backtest: Cumulative Return of the Strategy on BTC")
plt.xlabel("Date")
plt.ylabel("Cumulative Return")
plt.legend()
plt.tight_layout()
plt.show()

## Cumulative Return of my strategy

In [ ]:
# Reassigning appropriate column names to the dataset
btc_df.columns = ['Date', 'Price', 'Market_Cap', 'Total_Volume']

# Now, let's preprocess the data using these new column names
btc_df['Date'] = pd.to_datetime(btc_df['Date'])
btc_df.set_index('Date', inplace=True)

# Adjust the rest of the code to use the correct column names
btc_df['price'] = btc_df['Price']  # Aligning with the column expected by the strategy

# Now apply the strategy as before
btc_strategy_df = apply_trading_strategy(btc_df)

# Visualize the cumulative returns
plt.figure(figsize=(14, 8))
plt.plot(btc_strategy_df['cumulative_returns'], label='Cumulative Returns', color='teal')
plt.title('Cumulative Returns of the Trading Strategy on BTC')
plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
plt.legend(loc='upper left')
plt.grid(True)
plt.show()

In [ ]:
# Redefine the necessary functions for the strategy

# Calculate the EMAs
def calculate_macd(df, short_ema=13, long_ema=34):
    df['ema_short'] = df['price'].ewm(span=short_ema, adjust=False).mean()
    df['ema_long'] = df['price'].ewm(span=long_ema, adjust=False).mean()
    df['macd'] = df['ema_short'] - df['ema_long']
    df['signal_line'] = df['macd'].ewm(span=9, adjust=False).mean()
    df['macd_histogram'] = df['macd'] - df['signal_line']
    return df

# Calculate ATR
def calculate_atr(df, atr_period=13):
    df['high_low'] = df['price'].rolling(window=atr_period).apply(lambda x: x.max() - x.min())
    df['atr'] = df['high_low'].ewm(span=atr_period, adjust=False).mean()
    return df

# Apply the strategy for long positions
def apply_trading_strategy(df):
    df = calculate_macd(df)
    df = calculate_atr(df)
    
    df['signal'] = 0
    df['stop_loss'] = np.nan
    df['take_profit'] = np.nan

    for i in range(1, len(df)):
        if df['macd_histogram'].iloc[i-1] < 0 and df['macd_histogram'].iloc[i] > 0:
            # Check if conditions are met
            if df['price'].iloc[i-2:i+1].min() < df['price'].iloc[i-2:i+1].min():
                # Entry Point
                entry_price = df['price'].iloc[i]
                df.at[df.index[i], 'signal'] = 1
                df.at[df.index[i], 'stop_loss'] = entry_price - df['atr'].iloc[i]
                df.at[df.index[i], 'take_profit'] = entry_price + 2 * (entry_price - df['atr'].iloc[i])
    
    # Calculate strategy returns
    df['position'] = df['signal'].replace(to_replace=0, method='ffill')
    df['strategy_returns'] = df['price'].pct_change() * df['position'].shift(1)
    df['cumulative_returns'] = (1 + df['strategy_returns']).cumprod()

    return df

# Apply the strategy to BTC dataset
btc_strategy_df = apply_trading_strategy(btc_df)

# Visualize the cumulative returns
plt.figure(figsize=(14, 8))
plt.plot(btc_strategy_df['cumulative_returns'], label='Cumulative Returns', color='teal')
plt.title('Cumulative Returns of the Trading Strategy on BTC')
plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
plt.legend(loc='upper left')
plt.grid(True)
plt.show()
